In [ ]:
pip install --upgrade chromedriver-autoinstaller

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium

# Install xvfb
apt install -y xvfb

# Install Selenium-Profiles
pip uninstall -y selenium_profiles
pip install --no-cache-dir selenium_profiles>=2.2.6



# install python packages
pip install google-colab-shell
pip install webdriver-manager
pip install Pyvirtualdisplay

In [ ]:
!pip install pyap
!pip install usaddress

In [ ]:
import pandas as pd
import usaddress
import pyap
import requests
import time
import signal
import threading
import multiprocessing
import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.filterwarnings("ignore", category=InsecureRequestWarning)
from bs4 import BeautifulSoup
from requests.exceptions import SSLError

# Read the list of company websites from the parquet file
websites_df = pd.read_parquet('list of company websites.snappy.parquet')
print(len(websites_df))
for i in range(0,len(websites_df)):
  print(str(i)+" "+websites_df.iloc[i][0])


In [49]:


def scrape_contact_page(url):
    from selenium_profiles.webdriver import Chrome
    from selenium_profiles.profiles import profiles
    from selenium.webdriver.common.by import By  # locate elements
    from selenium_profiles.utils.colab_utils import display, showscreen, show_html # virtual display
    from webdriver_manager.chrome import ChromeDriverManager
    time.sleep(0.01)

    try:
        response = requests.get(url, verify=False)
    except SSLError as e:
        return "SSL Error"
    except requests.exceptions.RequestException as e:
        return "Request Exception"

    soup = BeautifulSoup(response.content, 'html.parser')
    anchor_tags = soup.find_all('a')

    contact_url = None
    for tag in anchor_tags:
        href = tag.get('href')
        if href is not None and ('contact' in href or 'connect-with-us' in href or 'kontakt' in href):
            contact_url = href
            break
    #try with selenium for js web pages
    if contact_url == None:
      from selenium_profiles.webdriver import Chrome
      from selenium_profiles.profiles import profiles
      from selenium.webdriver.common.by import By  # locate elements
      from selenium_profiles.utils.colab_utils import display, showscreen, show_html # virtual display

      from webdriver_manager.chrome import ChromeDriverManager
      my_display=None
      try:
        chromedriver_path = ChromeDriverManager(version="90.0.4430.24").install()

        profile = profiles.Windows()
        profile["cdp"]["cores"] = None

        mydriver = Chrome(profile, executable_path=chromedriver_path)

        my_display = display()
        my_display.start_display()
        driver = None
        try:
              driver = mydriver.start()
              driver.get(url)
              page_source = driver.page_source
              visible_text = driver.find_element(By.TAG_NAME, 'body').text
              links = driver.find_elements(By.TAG_NAME, 'a')
              for link in links:
                  href = link.get_attribute('href')
                  if href is not None and ('contact' in href or 'connect-with-us' in href or 'kontakt' in href):
                    contact_url = href
                    break
        except Exception as e:
           print(f"WebDriver Exception: {str(e)}")
        finally:
           if driver is not None:
              driver.quit()
      except Exception as e:
         #print(f"Setup Exception: {str(e)}")
         pass
      finally:
        if my_display is not None:
          my_display.stop_display()
    if contact_url == None:
      return url
    else:
      return contact_url


In [36]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [ ]:
# Print some links
for i in range(102,110):
  try:
    with time_limit(10):
      arg="https://"+websites_df.iloc[i][0]+"/"
      contact_page=None
      contact_page=scrape_contact_page(arg)


      if contact_page != None and not("SSL Error" in contact_page) and not("Request Exception" in contact_page):
        if "https://" not in contact_page:
            contact_page=arg+contact_page
      print(i)
      print(contact_page)
  except TimeoutException as e:
    print("Timed out!")

In [40]:
import re

In [50]:
def selenium_content(url):
      from selenium_profiles.webdriver import Chrome
      from selenium_profiles.profiles import profiles
      from selenium.webdriver.common.by import By
      from selenium_profiles.utils.colab_utils import display, showscreen, show_html

      from webdriver_manager.chrome import ChromeDriverManager
      visible_text=""
      my_display= None
      try:
        chromedriver_path = ChromeDriverManager(version="90.0.4430.24").install()

        profile = profiles.Windows()
        profile["cdp"]["cores"] = None

        mydriver = Chrome(profile, executable_path=chromedriver_path)

        my_display = display()
        my_display.start_display()
        driver = None
        try:
              driver = mydriver.start()
              driver.get(url)  # test client hints
              page_source = driver.page_source
              visible_text = driver.find_element(By.TAG_NAME, 'body').text
        except Exception as e:
           print(f"WebDriver Exception: {str(e)}")
        finally:
           if driver is not None:
              driver.quit()
      except Exception as e:
         #print(f"Setup Exception: {str(e)}")
         pass
      finally:
        if my_display is not None:
          my_display.stop_display()

      return visible_text

In [47]:

def scrape_addressUS(url):
    time.sleep(0.01)
    try:
        response = requests.get(url, verify=False)

    except SSLError as e:

        return "SSL Error"
    except requests.exceptions.RequestException as e:

        return "Request Exception"


    soup = BeautifulSoup(response.content, 'html.parser')

    def extract_address(text):
      address_pattern = r"(\d+)\s*([A-Za-z\s,.]+)\s*([A-Z]{2})\s+(\d{5})"
      addresses = re.findall(address_pattern, text)
      if addresses:
        address_parts = addresses[0]
        address = ", ".join(address_parts)
        return address
      else:
        return None


    soupText = soup.get_text()
    address=extract_address(soupText)
    address1 = pyap.parse(soupText, country='US')

    string=""
    if address:
      string+=address
      string+="//"

    if address1:
      string+=str(address1)
    if not address and not address1:
        seleniumText= selenium_content(url)
        address2=extract_address(seleniumText)
        address3 = pyap.parse(seleniumText, country='US')
        if address2:
          string+=address2
          string+='/'

        if address3:
          string+=str(address3)
    return string

In [43]:
def scrape_addressGer(url):
    time.sleep(0.01)
    try:
        response = requests.get(url, verify=False)
    except SSLError as e:
        return "SSL Error"
    except requests.exceptions.RequestException as e:
        return "Request Exception"

    soup = BeautifulSoup(response.content, 'html.parser')

    def extract_address(text):
      address_pattern =  r"([A-Za-zÄäÖöÜüß.,-]+)\s([A-Za-zÄäÖöÜüß.,-]+)([\d\s/-]+)\s*(\d{5})\s([A-Za-zÄäÖöÜüß,.]+)"
      addresses = re.findall(address_pattern, text)


      if addresses:
        address_parts = addresses[0]
        address = " ".join(address_parts)
        return address
      else:
        return None

    address=extract_address(soup.get_text())
    string=""
    if address:
      return address

    if not address:
        address2=extract_address(selenium_content(url))
        if address2:

          return address2

    return string

In [44]:
def scrape_addressUK(url):
    time.sleep(0.01)
    try:
        response = requests.get(url, verify=False)

    except SSLError as e:

        return "SSL Error"
    except requests.exceptions.RequestException as e:

        return "Request Exception"


    soup = BeautifulSoup(response.content, 'html.parser')

    address1 = None
    address1 = pyap.parse(soup.get_text(), country='GB')


    string = ""
    if address1:

       return str(address1)
    if not address1:
        address3 = pyap.parse(selenium_content(url), country='GB')
        if address3:

          return str(address3)

    return string

In [ ]:
us=850
ger=1622
file_path = 'output.txt'
file = open(file_path, 'w')

#Scrape US addresses
for i in range(0,us):
  try:
    with time_limit(10):
      arg="https://"+websites_df.iloc[i][0]+"/"

      contact_page=None
      contact_page=scrape_contact_page(arg)
      if contact_page != None and not("SSL Error" in contact_page) and not("Request Exception" in contact_page):
        if arg not in contact_page:
          if "https://" not in contact_page:
            contact_page=arg+contact_page
      print(i)
      if contact_page != None:
        address=scrape_addressUS(str(contact_page))
        file.write(str(i))
        file.write("\n")
        file.write(address)
        file.write("\n")
        print(address)
  except TimeoutException as e:
    print("Timed out!")

#Scrape German addresses
for i in range(us,ger):
  try:
    with time_limit(10):
      arg="https://"+websites_df.iloc[i][0]+"/"

      contact_page=None
      contact_page=scrape_contact_page(arg)
      if contact_page != None and not("SSL Error" in contact_page) and not("Request Exception" in contact_page):
        if arg not in contact_page:
          if "https://" not in contact_page:
            contact_page=arg+contact_page
      print(i)
      if contact_page != None:
        address=scrape_addressGer(str(contact_page))
        file.write(str(i))
        file.write("\n")
        file.write(address)
        file.write("\n")
        print(address)
  except TimeoutException as e:
    print("Timed out!")

#Scrape UK addresses
for i in range(ger,len(websites_df)):
  try:
    with time_limit(10):
      arg="https://"+websites_df.iloc[i][0]+"/"

      contact_page=None
      contact_page=scrape_contact_page(arg)
      if contact_page != None and not("SSL Error" in contact_page) and not("Request Exception" in contact_page):
        if arg not in contact_page:
          if "https://" not in contact_page:
            contact_page=arg+contact_page
      print(i)
      if contact_page != None:
        address=scrape_addressUK(str(contact_page))
        file.write(str(i))
        file.write("\n")
        file.write(address)
        file.write("\n")
        print(address)
  except TimeoutException as e:
    print("Timed out!")

file.close()